# Match Naturalis Collectors to Wikidata Items Using *Cosine Similarity*, `eventDate` Involved.

In this example we add `eventDate` of the source data, when the sample/occurrence was collected, to have a time reference, when the collector should have been  alive.

Basically we …

- match of `canonical_string` of WikiData to `canonical_string` of the source collectors (abbreviated names and full names, if given), and
- parse collector source names beforehand to get individual names out of name lists in the source data, we have used <https://libraries.io/rubygems/dwc_agent>, and in general we
- follow the example of Niels Klazenga <https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb>
- write the output to provide a DarwinCore attribution structure (for `verbatimName` we would need the `source_data` name(s))

Technical Notes — Review Code perhaps:
- TODO review score calculation of the matching of relating eventData with range of yob, yod
- TODO review DwC agent output, keep at this time custom columns for filter-sort-evaluation convenience
- (NN ⇌ Cosine) refactor relation: wd_matchtest ⇌ wikidata_unique (replaced wd_matchtest → wikidata_unique)

### Load Wikidata Data Set

Construct data using Jupyter Notebook [create_wikidata_datasets_botanists.ipynb](./create_wikidata_datasets_botanists.ipynb)

Out of the Wikidata items data set we create a data frame with unique canonical name strings and their counts.

In [1]:
import pandas as pd
import pprint, time, os

wikidata = pd.read_csv(
    "data/wikidata_persons_botanists_20231030_1539.csv",
    index_col=0, low_memory=False,
    dtype={
        'yob':'Int32',
        'yod':'Int32',
        'wyb':'Int32',
        'wye':'Int32'
    }
)
pprint.pprint(wikidata.columns)
display(wikidata.head())

Index(['item', 'itemLabel', 'surname', 'initials', 'canonical_string',
       'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni',
       'abbr', 'bionomia_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link',
       'orcid_link', 'harv_link', 'ipni_link', 'bionomia_link'],
      dtype='object')


,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q66612,Friedrich August Marschall von Bieberstein,Bieberstein,F.A.M.v.,"Bieberstein, F.A.M.v.","Bieberstein, Friedrich August Marschall von",NaN,43340073,0000 0001 1630 5464,1373,...,Q66612,1768,1826,<NA>,<NA>,http://www.wikidata.org/wiki/Q66612,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/6129-1,https://bionomia.net/Q66612
1,http://www.wikidata.org/entity/Q66934,Hans Hermann Behr,Behr,H.H.,"Behr, H.H.","Behr, Hans Hermann",NaN,20328622,0000 0001 1604 8680,42741,...,Q66934,1818,1904,<NA>,<NA>,http://www.wikidata.org/wiki/Q66934,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/619-1,https://bionomia.net/Q66934
2,http://www.wikidata.org/entity/Q66661,Jacob Christian Schäffer,Schäffer,J.C.,"Schäffer, J.C.","Schäffer, Jacob Christian",NaN,47016953,0000 0000 8343 3899,1101,...,NaN,1718,1790,<NA>,<NA>,http://www.wikidata.org/wiki/Q66661,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12818-1,NaN
3,http://www.wikidata.org/entity/Q67003,Johann Friedrich Klotzsch,Klotzsch,J.F.,"Klotzsch, J.F.","Klotzsch, Johann Friedrich",NaN,20426762,0000 0001 1749 2732,135,...,Q67003,1805,1860,<NA>,<NA>,http://www.wikidata.org/wiki/Q67003,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/4855-1,https://bionomia.net/Q67003
4,http://www.wikidata.org/entity/Q66322,Franz Anton Menge,Menge,F.A.,"Menge, F.A.","Menge, Franz Anton",NaN,59847236,0000 0001 1653 0899,73782,...,NaN,1808,1880,<NA>,<NA>,http://www.wikidata.org/wiki/Q66322,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23266-1,NaN


In [2]:
# compile data having only unique canonical strings
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

display(wd_matchtest)
display(wd_matchtest_fullnames)

,canonical_string,item
,,count
0,"(-Walraevens), O.H.",1
1,"(Lindberg), H.v.R.",1
2,"(Magistad), I.K.",1
3,"(Tsybulskaya), M.P.N.",1
4,"(botaniker), B.P.",1
...,...,...
65517,"Яковлевич, С.В.",1
65518,"Яфрэмаў, А.Л.",1
65519,"ас-Сури, И.",1


,canonical_string_fullname,item
,,count
0,"(-Walraevens), O Heylen",1
1,"(Lindberg), Hildur von Rettig",1
2,"(Magistad), Inger Kaasa",1
3,"(Tsybulskaya), Maria Pavlovna Nagibina",1
4,"(botaniker), Bror Pettersson",1
...,...,...
68027,"Яковлевич, Скробишевский, Владислав",1
68028,"Яфрэмаў, Аляксандр Лаўрэнцьевіч",1
68029,"ас-Сури, Ибн",1


## Load Collectors Data Set

**Data sources:**

- Jupyter Notebook for [create_naturalis_gbif-occurrence_collectors_eventDate_dataset.ipynb](./create_naturalis_gbif-occurrence_collectors_eventDate_dataset.ipynb)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`

Technical notes:

- the corresponding objects, variable names of Nils’ python code were:
```
refactor df_avh = → = collectors
refactor df_avh['label'] = → = collectors['canonical_string_collector_parsed']
…
```

In [3]:
# unique names parsed already by ruby gem package: dwcagent

collectors = pd.read_csv("data/Naturalis_doi-10.15468-dl.uw8rxk/occurrence_recordedBy_eventDate_occurrenceIDs_20230913_parsed.tsv", sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.

# Out of bounds nanosecond timestamp: 1652-01-01T00:00:00
#  because date nanoseconds range limitations of pandas, see https://stackoverflow.com/a/69507200/1240387
#  work around: use datetime or using pd.Periode(…)
print("modify time using pd.Periode(…) to get it work also on very old dates...")
for col in ['eventDate_mean', 'eventDate_min', 'eventDate_max']:
    print("- convert", col, "to pd.Period(...) in collectors")
    collectors[col] = collectors[col].apply(
        lambda x: pd.Period(
            x, freq='S' if col.lower().endswith('mean') else 'D' # Seconds or Day level
        )
    ) # D=day level
    # see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-period-aliases
print("modifing done.")

collectors.sort_values(by=['family', 'given','occurrenceID_firstsample'], inplace=True)
collectors

modify time using pd.Periode(…) to get it work also on very old dates...
- convert eventDate_mean to pd.Period(...) in collectors
- convert eventDate_min to pd.Period(...) in collectors
- convert eventDate_max to pd.Period(...) in collectors
modifing done.


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
86130,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kampen PN van; A,parsed:Kampen PN van<SEP>A,cleaned:<SEP>A,1,https://data.biodiversitydata.nl/naturalis/spe...,1899-08-07 00:00:00,1899-08-07,1899-08-07
180474,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wendt T; Villalobos C A; A; Navarrete I,parsed:T. Wendt<SEP>A. Villalobos C<SEP>A<SEP>...,cleaned:T. Wendt<SEP>A. Villalobos C<SEP>A<SEP...,5,https://data.biodiversitydata.nl/naturalis/spe...,1981-12-26 00:00:00,1981-03-20,1983-05-18
172212,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown; A,parsed:A,cleaned:A,1,https://data.biodiversitydata.nl/naturalis/spe...,NaT,NaT,NaT
55251,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fuentes C; A; Rosa de la,parsed:C. Fuentes<SEP>A<SEP>Rosa de la,cleaned:C. Fuentes<SEP>A<SEP>,1,https://data.biodiversitydata.nl/naturalis/spe...,1997-02-01 00:00:00,1997-02-01,1997-02-01
143852,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Romero EM; Fuentes RE; A,parsed:E.M. Romero<SEP>R.E. Fuentes<SEP>A,cleaned:E.M. Romero<SEP>R.E. Fuentes<SEP>A,1,https://data.biodiversitydata.nl/naturalis/spe...,1996-07-07 00:00:00,1996-07-07,1996-07-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189361,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,Štepánek J; Trávnícek B,parsed:J. Štepánek<SEP>B. Trávnícek,cleaned:J. Štepánek<SEP>B. Trávnícek,2,https://data.biodiversitydata.nl/naturalis/spe...,1998-11-06 12:00:00,1992-05-15,2005-04-30
89846,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,Kirschnerová L; Kirschner J; Štepánek J,parsed:L. Kirschnerová<SEP>J. Kirschner<SEP>J....,cleaned:L. Kirschnerová<SEP>J. Kirschner<SEP>J...,1,https://data.biodiversitydata.nl/naturalis/spe...,1983-05-09 00:00:00,1983-05-09,1983-05-09
189346,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,Štepánek J; Jakl J,parsed:J. Štepánek<SEP>J. Jakl,cleaned:J. Štepánek<SEP>J. Jakl,1,https://data.biodiversitydata.nl/naturalis/spe...,2003-06-01 00:00:00,2003-06-01,2003-06-01
65694,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,Hadinec J; Kirschner J; Sourkova M; Štepánek J,parsed:J. Hadinec<SEP>J. Kirschner<SEP>M. Sour...,cleaned:J. Hadinec<SEP>J. Kirschner<SEP>M. Sou...,1,https://data.biodiversitydata.nl/naturalis/spe...,1978-05-17 00:00:00,1978-05-17,1978-05-17


### Check Composition of Parsed Collector Data

In [4]:
# TODO review code of abbreviated names and full name matching
criterion_fullnames = collectors.given.str.contains('^\w{3,}', na=False)
print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
collectors[criterion_fullnames]

Show collecors given name has (propably) a full name (6756 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
141189,A-M-V-J,Renier,NaN,NaN,NaN,NaN,NaN,NaN,Renier A-M-V-J,parsed:Renier A-M-V-J,cleaned:Renier A-M-V-J,1,https://data.biodiversitydata.nl/naturalis/spe...,NaT,NaT,NaT
75652,A-ts'ai,Hsieh,NaN,NaN,NaN,NaN,NaN,NaN,Hsieh A-ts'ai,parsed:Hsieh A-ts'ai,cleaned:Hsieh A-ts'ai,1,https://data.biodiversitydata.nl/naturalis/spe...,1929-05-21 00:00:00,1929-05-21,1929-05-21
162946,A. Kneucker T,Stuckert,NaN,in,NaN,NaN,NaN,NaN,Stuckert in A. Kneucker T,parsed:Stuckert in A. Kneucker T,cleaned:Stuckert in A. Kneucker T,4,https://data.biodiversitydata.nl/naturalis/spe...,1902-01-01 00:00:00,1902-01-01,1902-01-01
83422,AFle,Jolis,NaN,NaN,NaN,NaN,NaN,NaN,Jolis AFle,parsed:Jolis AFle,cleaned:Jolis AFle,420,https://data.biodiversitydata.nl/naturalis/spe...,1860-07-06 19:47:47,1800-01-01,1983-10-04
14090,Aaaa,Bellynck,NaN,NaN,NaN,NaN,NaN,NaN,Bellynck AAAA,parsed:Bellynck AAAA,cleaned:Bellynck Aaaa,6,https://data.biodiversitydata.nl/naturalis/spe...,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144648,le,Roux A.,NaN,NaN,NaN,NaN,NaN,NaN,Roux A le; Lloyd JW,parsed:Roux A. le<SEP>J.W. Lloyd,cleaned:Roux A. le<SEP>J.W. Lloyd,1,https://data.biodiversitydata.nl/naturalis/spe...,1985-03-09 00:00:00,1985-03-09,1985-03-09
144650,le,Roux A.,NaN,NaN,NaN,NaN,NaN,NaN,Roux A le; Ramsey M,parsed:Roux A. le<SEP>M. Ramsey,cleaned:Roux A. le<SEP>M. Ramsey,9,https://data.biodiversitydata.nl/naturalis/spe...,1977-12-16 21:20:00,1972-09-13,1978-09-13
163366,le,Sueur F.A.,NaN,NaN,NaN,NaN,NaN,NaN,Sueur FA le,parsed:Sueur F.A. le,cleaned:Sueur F.A. le,23,https://data.biodiversitydata.nl/naturalis/spe...,1977-01-09 19:12:00,1951-03-15,1981-04-01
167399,le,Testu G.M.P.C.,NaN,NaN,NaN,NaN,NaN,NaN,Testu GMPC le,parsed:Testu G.M.P.C. le,cleaned:Testu G.M.P.C. le,3,https://data.biodiversitydata.nl/naturalis/spe...,NaT,NaT,NaT


In [5]:
# check the name-parsed columns if they are empty or need to be considerd as data for matching or not
for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
    test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
    print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
    display(test_collectors.head().get(["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title"]))


----------------------------------------
show names with **particle** found 4802 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
162946,A. Kneucker T,Stuckert,NaN,in,NaN,NaN,NaN,NaN
68,Aa,H. A. van der,NaN,van,NaN,NaN,NaN,NaN
52,Aa,H.A.,NaN,van der,NaN,NaN,NaN,NaN
81,Aalst,Mdjm,NaN,van,NaN,NaN,NaN,NaN
89,Aanen,D.K.,NaN,van der,NaN,NaN,NaN,NaN



----------------------------------------
show names with **suffix** found 22 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
188594,Bakker,Zinderen,Sr.,NaN,NaN,NaN,NaN,NaN
62158,Gradstein,NaN,SR,van,NaN,NaN,NaN,NaN
62137,Gradstein,NaN,SR,van,NaN,NaN,NaN,NaN
89493,Leopold,King,III,NaN,NaN,NaN,NaN,NaN
159263,Maurit,Flora,II,NaN,NaN,NaN,NaN,NaN



----------------------------------------
show names with **dropping_particle** found 0 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title



----------------------------------------
show names with **appellation** found 1 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
19154,McCullogh,NaN,NaN,NaN,NaN,NaN,Mrs,NaN


Compile `canonical_string…` for the collector data we will later match the WikiData names with:

In [6]:
collectors['canonical_string_collector_parsed'] = (
  # use collectors.family only where given name has NA values, otherwise use family name + given name
  collectors.family.where(
      # condition
      collectors.given.isna(),
      # any other 
      # TODO improve the combined name for canonical_string_collector_parsed if any of the other dwc_parsed fields is not NaN
      other= (collectors.family + ", " + collectors.given)
  )
)

# add collectors.particle if particle has multiple words, like “van der” or “Reyna de”
criterion = collectors["particle"].str.contains("\w+ \w+", na=False)

collectors['canonical_string_collector_parsed'][criterion] = collectors[criterion].apply(
    lambda this_df: "{particle} {family}, {given}".format(
        particle=this_df["particle"], 
        family=this_df["family"], 
        given=this_df["given"]
    ), axis="columns"
)
criterion = collectors["particle"].str.contains("\w+ \w+", na=False)
display(collectors[criterion].head())

/tmp/ipykernel_19457/1270537426.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors['canonical_string_collector_parsed'][criterion] = collectors[criterion].apply(


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max,canonical_string_collector_parsed
52,Aa,H.A.,NaN,van der,NaN,NaN,NaN,NaN,Aa HA van der,parsed:H.A. van der Aa,cleaned:H.A. van der Aa,1172,https://data.biodiversitydata.nl/naturalis/spe...,1968-04-20 09:21:51,1953-01-01,2005-11-01,"van der Aa, H.A."
89,Aanen,D.K.,NaN,van der,NaN,NaN,NaN,NaN,Aanen DK; Heijden L van der,parsed:D.K. van der Aanen<SEP>L. Heijden,cleaned:D.K. van der Aanen<SEP>L. Heijden,1,https://data.biodiversitydata.nl/naturalis/spe...,1995-10-11 00:00:00,1995-10-11,1995-10-11,"van der Aanen, D.K."
92,Aart,P.J.M.,NaN,van der,NaN,NaN,NaN,NaN,Aart PJM van der,parsed:P.J.M. van der Aart,cleaned:P.J.M. van der Aart,2,https://data.biodiversitydata.nl/naturalis/spe...,1957-11-22 00:00:00,1951-05-05,1964-06-11,"van der Aart, P.J.M."
542,Abdullah,A.,NaN,van der,NaN,NaN,NaN,NaN,Abdullah A; Shah A; Zon APM van der,parsed:A. van der Abdullah<SEP>A. Shah<SEP>A.P...,cleaned:A. van der Abdullah<SEP>A. Shah<SEP>A....,7,https://data.biodiversitydata.nl/naturalis/spe...,1979-09-03 17:08:34,1979-09-01,1979-09-05,"van der Abdullah, A."
545,Abdullah,A.,NaN,van der,NaN,NaN,NaN,NaN,Abdullah A; Shah M; Zon APM van der,parsed:A. van der Abdullah<SEP>M. Shah<SEP>A.P...,cleaned:A. van der Abdullah<SEP>M. Shah<SEP>A....,1,https://data.biodiversitydata.nl/naturalis/spe...,1979-09-04 00:00:00,1979-09-04,1979-09-04,"van der Abdullah, A."


In [7]:
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)

these_columns=["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title", 'canonical_string_collector_parsed']

if 'source_data' in collectors.columns:
    these_columns.append("source_data")

display(collectors.tail().get(these_columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data
189361,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Štepánek, J.",Štepánek J; Trávnícek B
89846,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Štepánek, J.",Kirschnerová L; Kirschner J; Štepánek J
189346,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Štepánek, J.",Štepánek J; Jakl J
65694,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Štepánek, J.",Hadinec J; Kirschner J; Sourkova M; Štepánek J
154410,Šumberová,K.,NaN,NaN,NaN,NaN,NaN,NaN,"Šumberová, K.",Simons ELAN; Šumberová K


In [8]:
# group and aggregate data to have unique name rows only for the matching of names later on
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    source_data=('source_data', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_firstsample', lambda x: list(x)[0]), # custom function, to get the first entry
    collectors_eventDate_mean=('eventDate_mean', 'mean'),
    collectors_eventDate_min=('eventDate_min', 'min'),
    collectors_eventDate_max=('eventDate_max', 'max')
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)

display(collectors_unique)

# column naming perhaps more clear (because we condensed the data)?
# collectors=collectors.add_suffix('_namegrouped') \
#  if not any(col.endswith("_namegrouped") for col in list(collectors.columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,Kampen PN van; A,18,https://data.biodiversitydata.nl/naturalis/spe...,1981-04-19 16:00:00,1899-08-07,1999-12-10
1,A'buino'o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A'buino'o,A'buino'o; Hunt PF,1,https://data.biodiversitydata.nl/naturalis/spe...,1965-08-24 00:00:00,1965-08-24,1965-08-24
2,A-M-V-J,Renier,NaN,NaN,NaN,NaN,NaN,NaN,"A-M-V-J, Renier",Renier A-M-V-J,1,https://data.biodiversitydata.nl/naturalis/spe...,NaT,NaT,NaT
3,A-ts'ai,Hsieh,NaN,NaN,NaN,NaN,NaN,NaN,"A-ts'ai, Hsieh",Hsieh A-ts'ai,1,https://data.biodiversitydata.nl/naturalis/spe...,1929-05-21 00:00:00,1929-05-21,1929-05-21
4,A. Kneucker T,Stuckert,NaN,in,NaN,NaN,NaN,NaN,"A. Kneucker T, Stuckert",Stuckert in A. Kneucker T,4,https://data.biodiversitydata.nl/naturalis/spe...,1902-01-01 00:00:00,1902-01-01,1902-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58303,Širjaev,G.I.,NaN,NaN,NaN,NaN,NaN,NaN,"Širjaev, G.I.",Širjaev GI; Sirjaev V,32,https://data.biodiversitydata.nl/naturalis/spe...,1927-05-29 22:53:29,1924-05-01,1932-09-26
58304,Šmite,D.,NaN,NaN,NaN,NaN,NaN,NaN,"Šmite, D.",Šmite D,13,https://data.biodiversitydata.nl/naturalis/spe...,1978-01-12 16:30:00,1975-01-01,1980-09-08
58305,Špacek,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Špacek, J.",Špacek J,2,https://data.biodiversitydata.nl/naturalis/spe...,1962-07-10 00:00:00,1962-07-10,1962-07-10
58306,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Štepánek, J.",Štepánek J; Kirschner J,620,https://data.biodiversitydata.nl/naturalis/spe...,1988-06-14 10:28:20,1966-05-25,2006-07-13


In [9]:
# show collectors with highest occurrenceID_collectors_count
collectors_unique.sort_values(
    by=['occurrenceID_collectors_count', 'family', 'given'], 
    ascending=[False, True, True]
).head(10)

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
5725,Boom,B.K.,NaN,NaN,NaN,NaN,NaN,NaN,"Boom, B.K.",Ooststroom SJ van; Boom BK,51929,https://data.biodiversitydata.nl/naturalis/spe...,1956-02-26 18:12:11,1856-01-01,1997-04-11
6579,Breteler,F.J.,NaN,NaN,NaN,NaN,NaN,NaN,"Breteler, F.J.",Maas PJM; Breteler FJ; Maas-van de Kamer H; Ni...,41442,https://data.biodiversitydata.nl/naturalis/spe...,1988-07-15 10:45:59,1955-06-12,2020-03-06
32761,Maxwell,J.F.,NaN,NaN,NaN,NaN,NaN,NaN,"Maxwell, J.F.",Maxwell JF; Sankamethawee W,38782,https://data.biodiversitydata.nl/naturalis/spe...,1996-08-29 12:11:24,1969-01-18,2013-04-11
26987,Koorders,S.H.,NaN,NaN,NaN,NaN,NaN,NaN,"Koorders, S.H.",Koorders SH; Valeton T,34147,https://data.biodiversitydata.nl/naturalis/spe...,1917-12-17 06:47:23,1829-08-27,2012-11-11
29050,Leeuwenberg,A.J.M.,NaN,NaN,NaN,NaN,NaN,NaN,"Leeuwenberg, A.J.M.",Sidiyasa K; Leeuwenberg AJM; Arbainsyah,32867,https://data.biodiversitydata.nl/naturalis/spe...,1973-07-14 13:58:10,1926-02-20,1999-11-16
48156,Soest,J.L.,NaN,NaN,NaN,NaN,NaN,NaN,"Soest, J.L.",Delanghe; Brakel KB van; Soest JL van,31684,https://data.biodiversitydata.nl/naturalis/spe...,1947-10-12 23:09:56,1803-08-10,1999-06-06
614,Ajgh,Kostermans,NaN,NaN,NaN,NaN,NaN,NaN,"Ajgh, Kostermans",Kostermans AJGH; Soegeng-Reksodihardjo W,30712,https://data.biodiversitydata.nl/naturalis/spe...,1959-02-23 21:53:36,1892-09-30,1994-11-15
55792,Wilde-Duyfjes,B.E.E.,NaN,NaN,NaN,NaN,NaN,NaN,"Wilde-Duyfjes, B.E.E.",Mennema J; Wilde-Duyfjes BEE de,29893,https://data.biodiversitydata.nl/naturalis/spe...,1986-10-15 13:20:06,1958-06-28,2019-09-04
57587,Itinere,Stud,NaN,biol Rheno-Trai in,NaN,NaN,NaN,NaN,"biol Rheno-Trai in Itinere, Stud",Stud biol Rheno-Trai in itinere; Krüger JHJ,28222,https://data.biodiversitydata.nl/naturalis/spe...,1963-03-31 08:11:10,1847-06-18,1996-07-08
55636,Wieringa,J.J.,NaN,NaN,NaN,NaN,NaN,NaN,"Wieringa, J.J.",Wieringa JJ,23167,https://data.biodiversitydata.nl/naturalis/spe...,2006-09-12 06:03:51,1980-08-19,2022-11-12


In [10]:
# TODO continue 2023-08-21 10:28:54
# collectors=collectors.add_suffix('_parsed') \
#  if not any(col.endswith("_parsed") for col in list(collectors.columns))

## Set Up the Cosine Similarity and Text Search

See 
- for the application code https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb
- for reading on the topic: Taylor, Josh. 2019. ‘Fuzzy Matching at Scale’. Towards Data Science (blog). 2 July 2019. https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536.

The `ngrams`-function is used as an analyzer in the text search later.

In [11]:
import pandas as pd, numpy as np, re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn # pip install sparse-dot-topn

def get_matches_df(sparse_matrix, A, B, top=100):
    non_zeros = sparse_matrix.nonzero()

    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]

    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similarity = np.zeros(nr_matches)

    for index in range(0, nr_matches):
        left_side[index] = A[sparserows[index]]
        right_side[index] = B[sparsecols[index]]
        similarity[index] = round(sparse_matrix.data[index], 3)

    return pd.DataFrame({'namematch_source_data': left_side,
                         'namematch_resource_data': right_side,
                         'namematch_similarity': similarity})

!pip install ftfy
from ftfy import fix_text

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text

    @param string: the text string to perform the ngram splitting on
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.replace('.', ' ')
    string = string.title()  # normalise case - capital at start of each word
    string = re.sub(' +', ' ', string).strip() # get rid of multiple spaces and replace with a single
    string = ' ' + string + ' '  # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    string = string.strip()
    this_ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in this_ngrams]

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


In [12]:
print("Show ngram examples:")
print("- simple name:", ngrams('Klazenga, N.'))
print("- data from collectors:", ngrams(collectors_unique["canonical_string_collector_parsed"].at[1])) 
print("- data from match-test:", ngrams(wd_matchtest['canonical_string'].at[0]))
print("- data from match-test (full name):", ngrams(wd_matchtest_fullnames['canonical_string_fullname'].at[3]))


Show ngram examples:
- simple name: ['Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N']
- data from collectors: ['Abu', 'bui', 'uin', 'ino', 'noo']
- data from match-test: ['Wal', 'alr', 'lra', 'rae', 'aev', 'eve', 'ven', 'ens', 'ns ', 's O', ' O ', 'O H']
- data from match-test (full name): ['Tsy', 'syb', 'ybu', 'bul', 'uls', 'lsk', 'ska', 'kay', 'aya', 'ya ', 'a M', ' Ma', 'Mar', 'ari', 'ria', 'ia ', 'a P', ' Pa', 'Pav', 'avl', 'vlo', 'lov', 'ovn', 'vna', 'na ', 'a N', ' Na', 'Nag', 'agi', 'gib', 'ibi', 'bin', 'ina']


In [13]:
# some example data
for i, row in enumerate(range(5)):
    if (i == 0):
        print('(WikiData’s) canonical_string = (constructed) canonical_string_fullname') 
    pprint.pprint("%s = %s" % (
        wd_matchtest['canonical_string'].at[row],
        wd_matchtest_fullnames['canonical_string_fullname'].at[row]
    ))


(WikiData’s) canonical_string = (constructed) canonical_string_fullname
'(-Walraevens), O.H. = (-Walraevens), O Heylen'
'(Lindberg), H.v.R. = (Lindberg), Hildur von Rettig'
'(Magistad), I.K. = (Magistad), Inger Kaasa'
'(Tsybulskaya), M.P.N. = (Tsybulskaya), Maria Pavlovna Nagibina'
'(botaniker), B.P. = (botaniker), Bror Pettersson'


In [14]:
def calculateTFIDFmatchingOfData(query_data, match_data, cossim_ntop=1, cossim_lower_bound=0.5):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with awesome_cossim_topn() and return matched data

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param cossim_ntop: how many cossim matches each shall be calculated (default 1, i.e. the highest similarity) — increase it to get more alternative
        matches with less similarity
    @param cossim_lower_bound: where is the lower similarity cut off to regard data as similar (default 0.5)

    @requires get_get_matches_df()
    @requires ngrams()
    @requires awesome_cossim_topn()
    @requires TfidfVectorizer()

    @return: a data frame dictionary: namematch_source_data, namematch_resource_data, namematch_similarity (from @see get_matches_df())
    @rtype pd.DataFrame
    """

    import time
    time_start = time.time()

    # Vectorize Wikidata name (use fit_transform())
    print('Vectorizing data. This may take a while...')
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
    tf_idf_matrix_clean = vectorizer.fit_transform(match_data)
    # Vectorize collectors’ names (use transform())
    tf_idf_matrix_dirty = vectorizer.transform(query_data)

    duration = time.time() - time_start
    print('Vectorizing completed: created a matrix of TF-IDF features after %s s' % duration)

    # Calculate Cosine Similarity; keep only the best match (ntop=1) and only if the similarity is greater than 0.5 (lower_bound=0.5)
    # (lower_bound: a threshold that the element of A*B must be greater than
    #  https://github.com/ing-bank/sparse_dot_topn/blob/3f40611b0553b50c27f23c7dcffc3ca9a9e8f5b5/sparse_dot_topn/awesome_cossim_topn.py#L26C9-L26C78)
    cossim_matches = awesome_cossim_topn(
        tf_idf_matrix_dirty,
        tf_idf_matrix_clean.transpose(),
        ntop=cossim_ntop,
        lower_bound=cossim_lower_bound
    )
    print("Cossim matches calculated after %s s" % (time.time() - time_start))

    print("Get all matches together ...")
    # construct the matching data frame
    matches_df = get_matches_df(
        cossim_matches,
        query_data,
        match_data,
        top=0
    )
    print("Done. Matches calculated after %s s" % (time.time() - time_start))

    return matches_df

In [15]:
criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values

matches = calculateTFIDFmatchingOfData(
    collectors_names, 
    wd_matchtest['canonical_string'], 
    cossim_ntop=1 # e.g. cossim_ntop=3 would give more alternative matches as well, having lower similarities, data would increase 3 times as well
)
matches = matches.sort_values(by=['namematch_similarity'], ascending=[False])
matches = matches.reset_index(names=['old_index'])
display(matches)

Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 6.799795866012573 s
Cossim matches calculated after 9.502326011657715 s
Get all matches together ...
Done. Matches calculated after 9.889939069747925 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,43072,"Šumberová, K.","Šumberová, K.",1.0
1,30302,"Prahl, P.","Prahl, P.",1.0
2,11544,"Florens, D.","Florens, D.",1.0
3,30320,"Pratt, A.E.","Pratt, A.E.",1.0
4,30317,"Prata, A.P.","Prata, A.P.",1.0
...,...,...,...,...
43068,37627,"Taris, D.","Morris, D.",0.5
43069,33822,"Savanh, P.","Lanorsavanh, S.",0.5
43070,1100,"Anson, R.J.","Watson, R.J.",0.5
43071,38382,"Ton, A.M.","Paton, A.M.",0.5


In [16]:
# criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values
matches_fullnames = calculateTFIDFmatchingOfData(
    collectors_fullnames, 
    wd_matchtest_fullnames['canonical_string_fullname'], 
    cossim_ntop=1 # 10 would give more alternative matches also with lesser similarity
)

matches_fullnames = matches_fullnames.sort_values(by=['namematch_similarity'], ascending=[False])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

display(matches_fullnames)

Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.67863392829895 s
Cossim matches calculated after 3.8679428100585938 s
Get all matches together ...
Done. Matches calculated after 3.8751466274261475 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,536,"Tadesse, Mesfin","Tadesse, Mesfin",1.000
1,260,"Hang, Sun","Hang, Sun",1.000
2,614,"Zhou, Hang","Zhou, Hang",1.000
3,100,"Chunghee, Lee","Chunghee, Lee",1.000
4,99,"Chung, Shih-Wen","Chung, Shih Wen",1.000
...,...,...,...,...
634,499,"Seruji, Ghazalli","Ismail, Ghazally",0.502
635,304,"Jgan, Vries","Vries, Jan de",0.501
636,414,"Nancy, Jard Bot","Davis, Nancy Jane",0.501
637,637,"van der Oudemans, Caja","Oudemans, Peter (V )",0.501


### Create Output Results

Combine the matches data frame back to the (Naturalis) collectors and Wikidata items …

Note: merging 18.770.000 collector matches earlier to wikidata was too much to calculate. Hence the descision was to make the data unique by canonical_string_collector_parsed.

In [17]:
# # join (only) abbreviated name matches with collector source data
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data', 
    how='left'
)

collectors_matches.dropna(subset=['namematch_similarity'], inplace=True)
display(collectors_matches) # 42298 rows × 18 columns

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
6,Aa,H. A. van der,NaN,van,NaN,NaN,NaN,NaN,"Aa, H. A. van der",Aa HA van der; Ooststroom SJ van,2,https://data.biodiversitydata.nl/naturalis/spe...,1962-07-07 00:00:00,1962-07-07,1962-07-07,0.0,"Aa, H. A. van der","Aa, H.A.v.d.",0.609
7,Aa,H.A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aa, H.A.",Gams KW; Aa HA van der,3,https://data.biodiversitydata.nl/naturalis/spe...,1973-02-06 08:00:00,1967-05-31,1977-08-27,1.0,"Aa, H.A.","Aa, H.A.v.d.",0.803
11,Aaku,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaku, A.",Aaku A,3,https://data.biodiversitydata.nl/naturalis/spe...,1909-11-19 16:00:00,1909-05-28,1910-07-29,2.0,"Aaku, A.","Karakuş, T.",0.562
13,Aalders,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aalders, A.",Sterk AA; Aalders A,1,https://data.biodiversitydata.nl/naturalis/spe...,1975-06-20 00:00:00,1975-06-20,1975-06-20,3.0,"Aalders, A.","Aalders, L.E.",0.798
14,Aalders,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Aalders, R.",Aalders R,2,https://data.biodiversitydata.nl/naturalis/spe...,1966-06-15 12:00:00,1966-05-31,1966-07-01,4.0,"Aalders, R.","Aalders, L.E.",0.792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58303,Širjaev,G.I.,NaN,NaN,NaN,NaN,NaN,NaN,"Širjaev, G.I.",Širjaev GI; Sirjaev V,32,https://data.biodiversitydata.nl/naturalis/spe...,1927-05-29 22:53:29,1924-05-01,1932-09-26,43068.0,"Širjaev, G.I.","Širjaev, G.I.",1.000
58304,Šmite,D.,NaN,NaN,NaN,NaN,NaN,NaN,"Šmite, D.",Šmite D,13,https://data.biodiversitydata.nl/naturalis/spe...,1978-01-12 16:30:00,1975-01-01,1980-09-08,43069.0,"Šmite, D.","Šmite, D.",1.000
58305,Špacek,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Špacek, J.",Špacek J,2,https://data.biodiversitydata.nl/naturalis/spe...,1962-07-10 00:00:00,1962-07-10,1962-07-10,43070.0,"Špacek, J.","Machacek, J.E.",0.609
58306,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Štepánek, J.",Štepánek J; Kirschner J,620,https://data.biodiversitydata.nl/naturalis/spe...,1988-06-14 10:28:20,1966-05-25,2006-07-13,43071.0,"Štepánek, J.","Štěpánek, J.",0.599


In [18]:
# join (only) full name matches with collector source data
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed' , right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

display(collectors_matches_fullname) # 628 rows × 18 columns

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,A-ts'ai,Hsieh,NaN,NaN,NaN,NaN,NaN,NaN,"A-ts'ai, Hsieh",Hsieh A-ts'ai,1,https://data.biodiversitydata.nl/naturalis/spe...,1929-05-21 00:00:00,1929-05-21,1929-05-21,0,"A-ts'ai, Hsieh","Hsieh, A Tsai",0.771
1,A. Kneucker T,Stuckert,NaN,in,NaN,NaN,NaN,NaN,"A. Kneucker T, Stuckert",Stuckert in A. Kneucker T,4,https://data.biodiversitydata.nl/naturalis/spe...,1902-01-01 00:00:00,1902-01-01,1902-01-01,1,"A. Kneucker T, Stuckert","Kneucker, Johann Andreas",0.516
2,Abbas,Damsah,NaN,NaN,NaN,NaN,NaN,NaN,"Abbas, Damsah",Damsah Abbas,1,https://data.biodiversitydata.nl/naturalis/spe...,1967-06-20 00:00:00,1967-06-20,1967-06-20,2,"Abbas, Damsah","Abbas, Alia",0.510
3,Abdela,Ahmet,NaN,NaN,NaN,NaN,NaN,NaN,"Abdela, Ahmet",Friis I; Bidgood GS; Assefa Hailu; Ermias Geta...,113,https://data.biodiversitydata.nl/naturalis/spe...,2007-10-20 00:12:44,2007-10-16,2007-10-24,3,"Abdela, Ahmet","İlçim, Ahmet",0.533
4,Abdullah A,Samat,NaN,bin,NaN,NaN,NaN,NaN,"Abdullah A, Samat",Samat bin Abdullah A; Turnau EA,110,https://data.biodiversitydata.nl/naturalis/spe...,1965-08-14 14:02:24,1961-08-03,2005-04-30,4,"Abdullah A, Samat","Abdullah, N",0.679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,Brink J,Bakhuizen,NaN,van den,NaN,NaN,NaN,NaN,"van den Brink J, Bakhuizen",Wilde WJJO de; Wilde-Duyfjes BEE de; Wilde JJF...,1,https://data.biodiversitydata.nl/naturalis/spe...,2007-04-01 00:00:00,2007-04-01,2007-04-01,634,"van den Brink J, Bakhuizen","Jr, Reinier Cornelis Bakhuizen van den Brink,",0.665
635,Brink S,Bakhuizen,NaN,van den,NaN,NaN,NaN,NaN,"van den Brink S, Bakhuizen",Wilde WJJO de; Wilde-Duyfjes BEE de; Wilde JJF...,1,https://data.biodiversitydata.nl/naturalis/spe...,2007-04-01 00:00:00,2007-04-01,2007-04-01,635,"van den Brink S, Bakhuizen","Jr, Reinier Cornelis Bakhuizen van den Brink,",0.654
636,Loo Hj,Brouwer-,NaN,van der,NaN,NaN,NaN,NaN,"van der Loo Hj, Brouwer-",Brouwer- van der Loo HJ,1,https://data.biodiversitydata.nl/naturalis/spe...,1970-06-24 00:00:00,1970-06-24,1970-06-24,636,"van der Loo Hj, Brouwer-","Brouwer, Henk",0.504
637,Oudemans,Caja,NaN,van der,NaN,NaN,NaN,NaN,"van der Oudemans, Caja",Oudemans CAJA; Sande Lacoste CM van der,8,https://data.biodiversitydata.nl/naturalis/spe...,1869-02-23 16:00:00,1860-06-26,1877-08-01,637,"van der Oudemans, Caja","Oudemans, Peter (V )",0.501


In [19]:
# join all name matches together
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_similarity', 'family'], ascending=[False, True], inplace=True)
collectors_all_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
22,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.",Aaronsohn A,3,https://data.biodiversitydata.nl/naturalis/spe...,1907-01-26 12:00:00,1906-12-06,1907-03-19,6.0,"Aaronsohn, A.","Aaronsohn, A.",1.0
48,Abbas,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbas, A.",Abdallah MS-A; Sa'ad FE-ZM; Mahdy M; Abbas A,378,https://data.biodiversitydata.nl/naturalis/spe...,1963-03-03 08:38:53,1936-02-11,1963-11-01,19.0,"Abbas, A.","Abbas, A.",1.0
53,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbe, E.C.",Abbe EC; Lampangi,537,https://data.biodiversitydata.nl/naturalis/spe...,1961-03-04 07:37:31,1932-01-01,1964-08-31,22.0,"Abbe, E.C.","Abbe, E.C.",1.0
56,Abbiatti,D.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbiatti, D.",Abbiatti D,2,https://data.biodiversitydata.nl/naturalis/spe...,1944-05-31 00:00:00,1937-10-01,1951-01-29,24.0,"Abbiatti, D.","Abbiatti, D.",1.0
67,Abbott,A.T.D.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, A.T.D.",Brand R; Bosch A; Abbott ATD,14,https://data.biodiversitydata.nl/naturalis/spe...,2002-12-17 01:36:00,1997-02-13,2010-05-27,31.0,"Abbott, A.T.D.","Abbott, A.T.D.",1.0


In [20]:
# # Save the plain name matching results only ...
# 
# if not os.path.exists('data'):
#     print("Make data directory for saving …")
#     os.makedirs('data')
# 
# # Set some global varialbes
# # this_timestamp_for_data=time.strftime('%Y%m%d') # 20230913
# this_timestamp_for_data=20231030
# 
# this_output_file='data/results_naturalis_collectors_vs_wikidata-botanists_cossim-similarity_plain-names_%s.csv' % (
#     this_timestamp_for_data
# )
# 
# collectors_all_matches.to_csv(this_output_file)
# 
# print("Wrote plain name matches of collector names into %s (%d kB)" % 
#     (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
# )

In [21]:
# merge now with WikiData: the matching data and the wiki data’s on the conaonical string name
collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

In [22]:
print("Show some name match examples (e.g. «Louis…» matching various names) …")
for testname in ['Louis', 'Abbot']:
    criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].map(lambda x: x.startswith(testname))    
    this_table=collectors_matches_g1_merged_wikidata[criterion].get([
        # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
        'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
        'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
        # 'canonical_string_fullname', 
        'itemLabel', 'wikidata_link',
        'collectors_eventDate_min', 'collectors_eventDate_max',
        'yob', 'yod', 'wyb', 'wye'
    ]).sort_values(by=['namematch_similarity'], ascending=[False])
    print("# ---------------------------------------------\n# «%s…» as test name, %d collector names begin with:" % (testname, criterion.sum()))    
    display(this_table)

Show some name match examples (e.g. «Louis…» matching various names) …
# ---------------------------------------------
# «Louis…» as test name, 13 collector names begin with:


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_similarity,itemLabel,wikidata_link,collectors_eventDate_min,collectors_eventDate_max,yob,yod,wyb,wye
20321,2,https://data.biodiversitydata.nl/naturalis/spe...,"Louis, A.","Louis, A.",1.000,A. Louis,http://www.wikidata.org/wiki/Q33682458,NaT,NaT,<NA>,<NA>,<NA>,<NA>
28103,10542,https://data.biodiversitydata.nl/naturalis/spe...,"Louis, A.M.","Louis, A.M.",1.000,Adriaan M. Louis,http://www.wikidata.org/wiki/Q21338327,1969-04-10,2013-03-02,1944,<NA>,<NA>,<NA>
28104,3339,https://data.biodiversitydata.nl/naturalis/spe...,"Louis, J.L.P.","Louis, J.L.P.",1.000,Jean Laurent Prosper Louis,http://www.wikidata.org/wiki/Q5928759,1900-01-01,1998-05-17,1903,1947,<NA>,<NA>
28105,51,https://data.biodiversitydata.nl/naturalis/spe...,"Louis-Marie, P.",Louis-Marie,0.914,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1925-07-28,1953-07-08,1896,1978,<NA>,<NA>
20320,59,https://data.biodiversitydata.nl/naturalis/spe...,Louis,"Louis, A.",0.868,A. Louis,http://www.wikidata.org/wiki/Q33682458,1904-05-28,1984-09-21,<NA>,<NA>,<NA>,<NA>
20324,3,https://data.biodiversitydata.nl/naturalis/spe...,"Louis, H.","Louis, A.",0.858,A. Louis,http://www.wikidata.org/wiki/Q33682458,1907-06-01,1953-10-01,<NA>,<NA>,<NA>,<NA>
20322,14,https://data.biodiversitydata.nl/naturalis/spe...,"Louis, F.","Louis, A.",0.857,A. Louis,http://www.wikidata.org/wiki/Q33682458,1910-01-01,1953-12-01,<NA>,<NA>,<NA>,<NA>
28106,1,https://data.biodiversitydata.nl/naturalis/spe...,"Louis-Marie, R.P.",Louis-Marie,0.857,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1934-07-09,1934-07-09,1896,1978,<NA>,<NA>
20326,4,https://data.biodiversitydata.nl/naturalis/spe...,"Louis, O.","Louis, A.",0.835,A. Louis,http://www.wikidata.org/wiki/Q33682458,1937-07-14,1937-07-27,<NA>,<NA>,<NA>,<NA>
20323,116,https://data.biodiversitydata.nl/naturalis/spe...,"Louis, F.H.","Louis, A.",0.755,A. Louis,http://www.wikidata.org/wiki/Q33682458,1853-08-03,1960-09-25,<NA>,<NA>,<NA>,<NA>


# ---------------------------------------------
# «Abbot…» as test name, 10 collector names begin with:


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_similarity,itemLabel,wikidata_link,collectors_eventDate_min,collectors_eventDate_max,yob,yod,wyb,wye
52,14,https://data.biodiversitydata.nl/naturalis/spe...,"Abbott, A.T.D.","Abbott, A.T.D.",1.000,A. T. D. Abbott,http://www.wikidata.org/wiki/Q117328147,1997-02-13,2010-05-27,1936,2013,<NA>,<NA>
53,2,https://data.biodiversitydata.nl/naturalis/spe...,"Abbott, E.K.","Abbott, E.K.",1.000,Edwin Kirk Abbott,http://www.wikidata.org/wiki/Q81587932,1889-01-01,1889-04-01,1840,1918,<NA>,<NA>
54,2,https://data.biodiversitydata.nl/naturalis/spe...,"Abbott, E.K.","Abbott, E.K.",1.000,Erwin Kirk Abbott,http://www.wikidata.org/wiki/Q113588322,1889-01-01,1889-04-01,1840,1918,<NA>,<NA>
56,10,https://data.biodiversitydata.nl/naturalis/spe...,"Abbott, W.L.","Abbott, W.L.",1.000,William Louis Abbott,http://www.wikidata.org/wiki/Q635604,1922-04-05,1922-04-30,1860,1936,<NA>,<NA>
55,106,https://data.biodiversitydata.nl/naturalis/spe...,"Abbott, I.A.","Abbott, I.",0.887,Isabella Abbott,http://www.wikidata.org/wiki/Q6077932,1946-05-01,1995-02-22,1919,2010,<NA>,<NA>
50,1,https://data.biodiversitydata.nl/naturalis/spe...,Abbott,"Abbott, G.",0.867,George Abbott,http://www.wikidata.org/wiki/Q47112598,NaT,NaT,<NA>,<NA>,<NA>,<NA>
51,1,https://data.biodiversitydata.nl/naturalis/spe...,"Abbott, D.P.","Abbott, G.",0.763,George Abbott,http://www.wikidata.org/wiki/Q47112598,1967-08-02,1967-08-02,<NA>,<NA>,<NA>,<NA>
41,1,https://data.biodiversitydata.nl/naturalis/spe...,"Abbot-Anderson, M.","Anderson, M.",0.631,Marilyn Anderson,http://www.wikidata.org/wiki/Q44754645,1933-06-21,1933-06-21,<NA>,<NA>,<NA>,<NA>
42,1,https://data.biodiversitydata.nl/naturalis/spe...,"Abbot-Anderson, M.","Anderson, M.",0.631,Mary Anderson,http://www.wikidata.org/wiki/Q111694258,1933-06-21,1933-06-21,1875,<NA>,<NA>,<NA>
43,1,https://data.biodiversitydata.nl/naturalis/spe...,"Abbot-Anderson, M.","Anderson, M.",0.631,Mark Anderson,http://www.wikidata.org/wiki/Q111990210,1933-06-21,1933-06-21,<NA>,<NA>,<NA>,<NA>


In [23]:
# # # # # # # # # # # # # # # # # # # # # # # # #
# display data and save custom columns
# # # # # # # # # # # # # # # # # # # # # # # # #
# pprint.pprint(collectors_matches_g1_merged_wikidata.columns)
# ## cell split - code
# collectors_matches_g1_merged_wikidata.head()
# ## cell split - code
# 
# # Select useful columns for data results
# collectors_wikidata_cossim = collectors_matches_g1_merged_wikidata[
#     ['canonical_string_collector_parsed', 'family', 'given', 
#      'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
#     'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
#     'item', 'canonical_string', 'itemLabel',
#     'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
#     'collectors_eventDate_mean', 'collectors_eventDate_min', 'collectors_eventDate_max',
#      'yob', 'yod', 'wyb'
#     ]
# ]
# 
# # Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
# collectors_wikidata_cossim.sort_values(by=['namematch_similarity', 'family', 'given'], ascending=[False, True, True], inplace=True)
# 
# collectors_wikidata_cossim # comparison-match of «Kotschy, Karl Georg Th» (collector data) →← «Kotschy, T» (Wikidata) has only 0.5 similarity but corresponds to the correct person name we need
# ## cell split - code
# 
# 
# # TODO further evaluation or filtering, counting, clean up aso.
# if not os.path.exists('data'):
#     os.makedirs('data')
# 
# # naturalis_collectors_cosine-similarity_wikidata-botanists_%s.csv
# this_output_file='data/results_naturalis_collectors_vs_wikidata-botanists_cossim-similarity_merged-data_%s.csv' % (
#     this_timestamp_for_data
# )
# 
# collectors_wikidata_cossim.to_csv(this_output_file)
# 
# print("Wrote matches of collector names into %s (%d kB)" % 
#     (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
# )

## Output Mapping to DarwinCore Attribution Output

Here we map table data fields to fields of DarwinCore Attribution (<https://github.com/tdwg/attribution/>, <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml>) 

## Scoring

Individual scored properties should actually be balanced in such a way that one can simply add up these different property scores; in this case, assessment of the calculated values is still necessary. The problem here with calculation with a distance measure is that we have the opposite of similarity, whose distance can become greater than 1, which must somehow be mapped to a scope of 0 … 1 (or -1 … 0 … 1) (TODO review).

General thoughts: With a score of -1 to 1, it can be assumed that:
* -1 means full devaluation or no agreement
* 1 means full upvoting or agreement, and
* 0 can have several interpretations: it is in between, or no rating possible, or missing values.

### Task to Be Solved in Evaluating the Life Time ~ Rating/Scoring

We have grouped the collection date (evenDate) to the name in the source data, so it may be that for (abbreviated) names, e.g. “Bachmann, F.”, the collection date is valid for *several* personal names, not just one. This must be taken into account when considering and evaluating whether the life data match the collection date. The rating of the life data has the following idea:

| Score (life time) | Remarks | 
|--|--|
| 1.0  | complete match                     |
| 0.5  | somewhat correct, but has errors or mistakes, indicating multiple person names    |
| 0.0   | no evaluation (or not possible) |
| -0.5 | is rather to be rejected, indicating multiple person names and possibly overlapping time spans of the collection date of different person names, or mistakes in the original data |
| -1.0 | completely rejected                |

### Task to Be Solved With Several Names ~ Assessment/Score

Since we do not know if there are other possible names somewhere when there is only one name, we cannot assign a “1” (= full agreement) with certainty, so it was decided that if only 1 name was found, this would be evaluated as zero, in the sense of no evaluation. So when evaluating the multiple names, only the mismatches are evaluated, according to the idea:

| Score (multiple names) | Remarks | 
|--|--|
| 1.0  | this value (=full upvoting or agreement) would never be set in this regard, since we do not know all the full names of the cosmos ;-), and could state this score certainty of 1.0 |
| 0.0 | no evaluation, because only 1 name found | 
| less than 0 | multiple names found, i.e. deduction (perhaps just -0.5, as a decision needs to be made) | 

---

TODO review interpretation:

- the fields are defined in <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml> and regarding from this DwC-attribution concept: is it correct to map it like the following (`name` would represent the *interpreted* resource name (in long format), not the *source* collector `name` (in (theoretically) long format))?
    ```
    name          ← itemLabel (wikiData)
    alternateName ← canonical_string_collector_parsed (actual collector name)
    ```

In [24]:
pprint.pprint(collectors_matches_g1_merged_wikidata.columns)

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'source_data', 'occurrenceID_collectors_count',
       'occurrenceID_collectors_firstsample', 'collectors_eventDate_mean',
       'collectors_eventDate_min', 'collectors_eventDate_max', 'old_index',
       'namematch_source_data', 'namematch_resource_data',
       'namematch_similarity', 'item', 'itemLabel', 'surname', 'initials',
       'canonical_string', 'canonical_string_fullname', 'orcid', 'viaf',
       'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb',
       'wye', 'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link',
       'bionomia_link'],
      dtype='object')


In [25]:
# TODO review score (similarity + ((yob,yod) ~ (eventDate_min, eventDate_max))

collectors_wikidata_cossim = collectors_matches_g1_merged_wikidata[
    ['canonical_string_collector_parsed', 'family', 'given', 
     'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
     'source_data',
    'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
    'item', 'canonical_string', 'itemLabel',
    'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
    'collectors_eventDate_mean', 'collectors_eventDate_min', 'collectors_eventDate_max',
     'yob', 'yod', 'wyb'
    ]
]

# Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
collectors_wikidata_cossim.sort_values(
    by=['namematch_similarity', 'family', 'given'], 
    ascending=[False, True, True], 
    inplace=True
)

dwcagent_attr_output=collectors_wikidata_cossim.get([
    "occurrenceID_collectors_firstsample", 
    "canonical_string_collector_parsed",
    'family', 'given',
    "namematch_similarity", 
    "source_data", 
    "itemLabel", 
    "item",
    "collectors_eventDate_min",
    "collectors_eventDate_max",
    'yob', 'yod'
]).copy()

dwcagent_attr_output['canonical_string_collector_parsed'].replace(to_replace=r'([^,]+),\s*(.+)', value='\\2 \\1', inplace=True, regex=True)
dwcagent_attr_output['namematch_similarity_annotation'] = dwcagent_attr_output['namematch_similarity'].astype(str).str.replace(r'(.+)', '\\1 (cos similarity)', regex=True)
# dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'namematch_similarity_annotation', '', allow_duplicates=True)

dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'life_time_periode', '', allow_duplicates=True)

combine_life_times = lambda this_df: ("%s-%s" % (this_df["yob"], this_df["yod"])).replace(r"<NA>", "?")
dwcagent_attr_output["life_time_periode"]=dwcagent_attr_output.apply(combine_life_times, axis="columns")

# dwcagent_attr_output["life_time_periode"]

years_from_birth_until_first_collection_activity = 10
dwcagent_attr_output["yob_is_lt_eventDate_min"] = dwcagent_attr_output["yob"] + years_from_birth_until_first_collection_activity < dwcagent_attr_output["collectors_eventDate_min"].dt.year
dwcagent_attr_output["yod_is_gt_eventDate_max"] = dwcagent_attr_output["yod"] > dwcagent_attr_output["collectors_eventDate_max"].dt.year
dwcagent_attr_output["custom_score_lifetime"] = 0
dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'custom_score_lifetime_annotation', '', allow_duplicates=True)

# df.loc[(df['column_of_interest'] … condition), 'fill_to_column'] = value 

dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"],
    "custom_score_lifetime"
] = 1
# True cases but <NA> missing values
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(),
    "custom_score_lifetime"
] = 1
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"],
    "custom_score_lifetime"
] = 1
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(),
    "custom_score_lifetime"
] = 0

# False cases
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime"
] = -1
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==True) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime"
] = 0.5
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == True),
    "custom_score_lifetime"
] = 0.5

# False cases but <NA> missing values
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull()),
    "custom_score_lifetime"
] = -0.5
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull()) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime"
] = -0.5

# annotations True cases
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"], 
    "custom_score_lifetime_annotation"
] = "full match"

# annotations True cases but <NA> missing values
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(), 
    "custom_score_lifetime_annotation"
] = "OK? year of death is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"], 
    "custom_score_lifetime_annotation"
] = "OK? year of birth is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(), 
    "custom_score_lifetime_annotation"
] = "unknown life time"

# annotations False cases
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False), 
    "custom_score_lifetime_annotation"
] = "life time not matching any enventDate (yob + %s … yod)" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==True) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False), 
    "custom_score_lifetime_annotation"
] = "OK yob + %s, but yod not matching, check name and liftime data" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == True), 
    "custom_score_lifetime_annotation"
] = "yob + %s not matching, OK yod, check name and liftime data" % years_from_birth_until_first_collection_activity
# annotations False cases but <NA> missing values
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull()), 
    "custom_score_lifetime_annotation"
] = "yob + %s not matching, yod unknown, check name and liftime data" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull()) & (dwcagent_attr_output["yod_is_gt_eventDate_max"]==False), 
    "custom_score_lifetime_annotation"
] = "yob unknown, yod not matching, check name and liftime data"

dwcagent_attr_output["custom_score_multiple_names"]=0 # 0 shall mean: we don’t know yet for real
dwcagent_attr_output.loc[
    (dwcagent_attr_output['canonical_string_collector_parsed'].duplicated(keep=False)),
    'custom_score_multiple_names'
] = -0.5 # one decision has to be made, so cut the range of -1 to 0 only into half (or include multiple count somehow?)

dwcagent_attr_output['custom_score_overall'] = (
    dwcagent_attr_output['namematch_similarity'] * \
    (
        ( dwcagent_attr_output["custom_score_lifetime"] + dwcagent_attr_output['custom_score_multiple_names']) / 2
    )
).round(3)

dwcagent_attr_output['attributionRemarks'] = dwcagent_attr_output.apply(
    lambda row: "{similarity_distance_note};"
                " {score_overall:.2f} (score overall);"
                " {lifetime_periode} (life time);"
                " {lifetime_score:.1f} (life time score);"
                " {lifetime_score_annote} (life time score note);"
                " {score_multinames:.2f} (score multiple names);"
        .format(
    similarity_distance_note=row['namematch_similarity_annotation'],
    lifetime_periode=row["life_time_periode"],
    lifetime_score=row["custom_score_lifetime"],
    lifetime_score_annote=row["custom_score_lifetime_annotation"],
    score_overall=row["custom_score_overall"],
    score_multinames=row["custom_score_multiple_names"]
    ), axis='columns'
)

# adjust dwcagent displayOrder also to olerall score
dwcagent_attr_output.sort_values(
    by=['namematch_similarity', 'family', 'given', 'custom_score_overall'], 
    ascending=[False, True, True, False], 
    inplace=True
)
# use ordered canonical_string_collector_parsed to generate displayOrder
temp_duplicated = dwcagent_attr_output['canonical_string_collector_parsed'].duplicated() 
    # duplicated() keeps the first value False and mark all other duplicats as True, i.e. we can cumulate the Trues, it gives the order index
temp_insert_value=temp_duplicated.groupby(dwcagent_attr_output['canonical_string_collector_parsed']).cumsum() + 1 # display order starts at 1, incrementing
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('canonical_string_collector_parsed') + 1, 'displayOrder', temp_insert_value, allow_duplicates=True)

# test an show example data
show_display_output=True
if show_display_output:
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == True].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_similarity",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime', 'custom_score_lifetime_annotation'
    ]).head(5))
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == False].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_similarity",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime', 'custom_score_lifetime_annotation'
    ]).head(5))

/tmp/ipykernel_19457/4017281334.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_wikidata_cossim.sort_values(


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_similarity,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime,custom_score_lifetime_annotation
9,A. Aaronsohn,Aaron Aaronsohn,0.50,1.0 (cos similarity); 0.50 (score overall); 18...,0.0,1.0,1876-1919,1906-12-06,1907-03-19,True,True,1.0,full match
32,E.C. Abbe,Ernst Cleveland Abbe,0.50,1.0 (cos similarity); 0.50 (score overall); 19...,0.0,1.0,1905-2000,1932-01-01,1964-08-31,True,True,1.0,full match
34,D. Abbiatti,Delia Abbiatti,0.50,1.0 (cos similarity); 0.50 (score overall); 19...,0.0,1.0,1918-?,1937-10-01,1951-01-29,True,<NA>,1.0,OK? year of death is missing
52,A.T.D. Abbott,A. T. D. Abbott,0.50,1.0 (cos similarity); 0.50 (score overall); 19...,0.0,1.0,1936-2013,1997-02-13,2010-05-27,True,True,1.0,full match
53,E.K. Abbott,Edwin Kirk Abbott,0.25,1.0 (cos similarity); 0.25 (score overall); 18...,-0.5,1.0,1840-1918,1889-01-01,1889-04-01,True,True,1.0,full match


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_similarity,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime,custom_score_lifetime_annotation
101,S.K. Abdullah,Samir K. Abdullah,-0.25,1.0 (cos similarity); -0.25 (score overall); 1...,0.0,1.0,1947-?,1921-03-01,1986-04-01,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
214,E. Acharius,Erik Acharius,0.25,1.0 (cos similarity); 0.25 (score overall); 17...,0.0,1.0,1757-1819,NaT,NaT,False,True,0.5,"yob + 10 not matching, OK yod, check name and ..."
228,G. Achoundong,Gaston Achoundong,-0.25,1.0 (cos similarity); -0.25 (score overall); 1...,0.0,1.0,1950-?,1960-04-21,2006-06-08,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
258,J.P.H. Acocks,John Phillip Harison Acocks,-0.50,1.0 (cos similarity); -0.50 (score overall); 1...,0.0,1.0,1911-1979,1855-05-04,1984-06-29,False,False,-1.0,life time not matching any enventDate (yob + 1...
343,A.M. Adamson,Alastair Martin Adamson,0.25,1.0 (cos similarity); 0.25 (score overall); 19...,0.0,1.0,1901-1945,NaT,NaT,False,True,0.5,"yob + 10 not matching, OK yod, check name and ..."


In [26]:
column_map_dwcagent_attr = {
    'occurrenceID_collectors_firstsample':'occurrenceID',
    'canonical_string_collector_parsed':  'alternateName',
    'source_data':                        'verbatimName',
    'itemLabel':                          'name',
    'item':                               'identifier',
    'collectors_eventDate_min':           'startedAtTime',
    'collectors_eventDate_max':           'endedAtTime',
    'namematch_similarity':               'custom_namematch_similarity'
}
dwcagent_attr_output.rename(
    mapper=column_map_dwcagent_attr,
    axis='columns',
    inplace=True)

dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'agentIdentifierType', 'wikidata' , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('agentIdentifierType') + 1, 'agentType'          , 'Person'   , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'action'             , 'collected', allow_duplicates=True)

show_display_output=False
if show_display_output:
    dwcagent_attr_output.head(20)

dwcagent_attr_output=dwcagent_attr_output.reindex(
    columns=[
        'occurrenceID', # no DwC agent standard (yet)?
        'verbatimName',
        'alternateName',
        'displayOrder', # shall start from 1, 2, 3 …
        'name',
        'attributionRemarks',
        'startedAtTime',
        'endedAtTime',
        'agentType',
        'action',
        'agentIdentifierType',
        'identifier',
        "custom_score_overall", # keep it for calculation convenience, no standard in DwC agent
        'custom_namematch_similarity',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_multiple_names',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_lifetime' # keep it for calculation convenience, no standard in DwC agent
    ]
)
# column deletion not neccessary after ….reindex(columns=[…])
# for this_column in ['yob', 'yod', 'life_time_periode', 'yob_is_lt_eventDate_min', 'yod_is_gt_eventDate_max', 'score_lifetime_annotation']:
#     del dwcagent_attr_output[this_column]


In [27]:
show_display_output=True
if show_display_output:
    criterion = dwcagent_attr_output['alternateName'].map(lambda x: x.startswith('S. Ahmad'))
    criterion = dwcagent_attr_output['custom_score_multiple_names'].map(lambda x: x < 0 )
    
    display(dwcagent_attr_output[criterion].head(20))

,occurrenceID,verbatimName,alternateName,displayOrder,name,attributionRemarks,startedAtTime,endedAtTime,agentType,action,agentIdentifierType,identifier,custom_score_overall,custom_namematch_similarity,custom_score_multiple_names,custom_score_lifetime
26,https://data.biodiversitydata.nl/naturalis/spe...,Abdallah MS-A; Sa'ad FE-ZM; Mahdy M; Abbas A,A. Abbas,1,Alia Abbas,1.0 (cos similarity); -0.25 (score overall); ?...,1936-02-11,1963-11-01,Person,collected,wikidata,http://www.wikidata.org/entity/Q60141229,-0.25,1.0,-0.5,0.0
27,https://data.biodiversitydata.nl/naturalis/spe...,Abdallah MS-A; Sa'ad FE-ZM; Mahdy M; Abbas A,A. Abbas,2,Abdulla Abbas,1.0 (cos similarity); -0.25 (score overall); ?...,1936-02-11,1963-11-01,Person,collected,wikidata,http://www.wikidata.org/entity/Q88804360,-0.25,1.0,-0.5,0.0
53,https://data.biodiversitydata.nl/naturalis/spe...,Abbott EK,E.K. Abbott,1,Edwin Kirk Abbott,1.0 (cos similarity); 0.25 (score overall); 18...,1889-01-01,1889-04-01,Person,collected,wikidata,http://www.wikidata.org/entity/Q81587932,0.25,1.0,-0.5,1.0
54,https://data.biodiversitydata.nl/naturalis/spe...,Abbott EK,E.K. Abbott,2,Erwin Kirk Abbott,1.0 (cos similarity); 0.25 (score overall); 18...,1889-01-01,1889-04-01,Person,collected,wikidata,http://www.wikidata.org/entity/Q113588322,0.25,1.0,-0.5,1.0
252,https://data.biodiversitydata.nl/naturalis/spe...,Ackermann M; et al.,M. Ackermann,1,Markus Ackermann,1.0 (cos similarity); 0.25 (score overall); 19...,2002-09-11,2005-10-01,Person,collected,wikidata,http://www.wikidata.org/entity/Q21504506,0.25,1.0,-0.5,1.0
253,https://data.biodiversitydata.nl/naturalis/spe...,Ackermann M; et al.,M. Ackermann,2,Marianne Ackermann,1.0 (cos similarity); -0.25 (score overall); ?...,2002-09-11,2005-10-01,Person,collected,wikidata,http://www.wikidata.org/entity/Q36619087,-0.25,1.0,-0.5,0.0
254,https://data.biodiversitydata.nl/naturalis/spe...,Ackermann M; et al.,M. Ackermann,3,Manfred Ackermann,1.0 (cos similarity); -0.25 (score overall); ?...,2002-09-11,2005-10-01,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112660,-0.25,1.0,-0.5,0.0
339,https://data.biodiversitydata.nl/naturalis/spe...,Blank D; Adams RP,R.P. Adams,1,Robert Phillip Adams,1.0 (cos similarity); 0.25 (score overall); 19...,1964-10-30,1996-11-29,Person,collected,wikidata,http://www.wikidata.org/entity/Q10363201,0.25,1.0,-0.5,1.0
340,https://data.biodiversitydata.nl/naturalis/spe...,Blank D; Adams RP,R.P. Adams,2,Robert Perry Adams,1.0 (cos similarity); 0.00 (score overall); 18...,1964-10-30,1996-11-29,Person,collected,wikidata,http://www.wikidata.org/entity/Q21504568,0.00,1.0,-0.5,0.5
338,https://data.biodiversitydata.nl/naturalis/spe...,Blank D; Adams RP,R.P. Adams,3,Robert Perry Adams,1.0 (cos similarity); -0.25 (score overall); ?...,1964-10-30,1996-11-29,Person,collected,wikidata,http://www.wikidata.org/entity/Q10363200,-0.25,1.0,-0.5,0.0


In [28]:
# TODO further evaluation or filtering, counting, clean up aso.
if not os.path.exists('data'):
    os.makedirs('data')

# this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
this_timestamp_for_data=20231030
this_output_file='data/results_naturalis_collectors-eventDate_vs_wikidata-botanists_cossim-similarity_dwc-agent-output_%s.csv' % (
    this_timestamp_for_data
)

dwcagent_attr_output.to_csv(this_output_file, index=False)

print("Wrote matches of collector names as dwc-agent-output into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote matches of collector names as dwc-agent-output into data/results_naturalis_collectors-eventDate_vs_wikidata-botanists_cossim-similarity_dwc-agent-output_20231030.csv (18099 kB)


## Documentation

TODO document columns

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
occurrenceID_collectors_count | count of all occurrenceID of one particular collector name
occurrenceID_collectors_firstsample | a data sample of an occurrenceID 
eventDate | date of the sampling event (required by GBIF, ☞ https://www.gbif.org/data-quality-requirements-sampling-events)
eventDate_min | calculated earliest date of all the sampling events within the data
eventDate_max | calculated latest date of all the sampling events within the data
eventDate_mean | calculated mean date of all the sampling events within the data
TODO activity_span | Number of years between first and last collection
**Name matching** |
namematch_source_data | matched name of the collector data set
namematch_resource_data | matched name of Wikidata the collector was tried to matched to
namematch_similarity | calculated cosine-similarity
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label (perhaps similar to the full name)
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))